In [1]:
# IMPORTS

In [2]:
import sys
import time
from pathlib import Path
from geopy.geocoders import Nominatim
import pygtfs
import os
from graph_tool.all import *
from pyrosm import get_data, OSM
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import shapely.geometry
from datetime import datetime, date, time, timedelta
import time as tm
from pyrosm.data import sources
from collections import defaultdict
from geopy.exc import GeocoderServiceError
import math
import folium
from IPython.display import display, SVG
from math import radians, cos, sin, asin, sqrt
import pandas as pd

In [3]:
# INFO GETTER

In [4]:
start = tm.time()
print("GETTING OSM INFO")

# PATHS
AVES_ROOT = Path("..")
EOD_PATH = AVES_ROOT / "data" / "external" / "EOD_STGO"
OSM_PATH = AVES_ROOT / "data" / "external" / "OSM"

## OSM ##

def get_osm_data():
    """
    Obtains the required OpenStreetMap data using the 'pyrosm' library. This gives the map info of Santiago.

    Returns:
        graph: osm data converted to a graph
    """
    # Download latest OSM data
    fp = get_data(
        "Santiago",
        update=True,
        directory=OSM_PATH
    )

    osm = OSM(fp)

    nodes, edges = osm.get_network(nodes=True)

    graph = Graph()

    # Create vertex properties for lon and lat
    lon_prop = graph.new_vertex_property("float")
    lat_prop = graph.new_vertex_property("float")
    
    # Create properties for the ids
    # Every OSM node has its unique id, different from the one given in the graph
    node_id_prop = graph.new_vertex_property("long")
    graph_id_prop = graph.new_vertex_property("long")
    
    # Create edge properties
    u_prop = graph.new_edge_property("long")
    v_prop = graph.new_edge_property("long")
    length_prop = graph.new_edge_property("double")
    weight_prop = graph.new_edge_property("double")

    vertex_map = {}

    print("GETTING OSM NODES...")
    for index, row in nodes.iterrows():
        lon = row['lon']
        lat = row['lat']
        node_id = row['id']
        graph_id = index
        node_coords[node_id] = (lat, lon)
        
        vertex = graph.add_vertex()
        vertex_map[node_id] = vertex
        
        # Assigning node properties
        lon_prop[vertex] = lon
        lat_prop[vertex] = lat
        node_id_prop[vertex] = node_id
        graph_id_prop[vertex] = graph_id

    # Assign the properties to the graph
    graph.vertex_properties["lon"] = lon_prop
    graph.vertex_properties["lat"] = lat_prop
    graph.vertex_properties["node_id"] = node_id_prop
    graph.vertex_properties["graph_id"] = graph_id_prop
    
    print("DONE")
    print("GETTING OSM EDGES...")

    for index, row in edges.iterrows():
        source_node = row['u']
        target_node = row['v']
        
        if row["length"] < 2 or source_node == "" or target_node == "":
            continue # Skip edges with empty or missing nodes
            
        if source_node not in vertex_map or target_node not in vertex_map:
            print(f"Skipping edge with missing nodes: {source_node} -> {target_node}")
            continue  # Skip edges with missing nodes
            
        source_vertex = vertex_map[source_node]
        target_vertex = vertex_map[target_node]
        
        if not graph.vertex(source_vertex) or not graph.vertex(target_vertex):
            print(f"Skipping edge with non-existent vertices: {source_vertex} -> {target_vertex}")
            continue  # Skip edges with non-existent vertices
            
        # Calculate the distance between the nodes and use it as the weight of the edge
        source_coords = node_coords[source_node]
        target_coords = node_coords[target_node]
        distance = abs(source_coords[0] - target_coords[0]) + abs(source_coords[1] - target_coords[1])
        
        e = graph.add_edge(source_vertex, target_vertex)
        u_prop[e] = source_node
        v_prop[e] = target_node
        length_prop[e] = row["length"]
        weight_prop[e] = distance
        
    graph.edge_properties["u"] = u_prop
    graph.edge_properties["v"] = v_prop
    graph.edge_properties["length"] = length_prop
    graph.edge_properties["weight"] = weight_prop

    print("OSM DATA HAS BEEN SUCCESSFULLY RECEIVED")
    return graph

# OSM Graph
node_coords = {}
osm_graph = get_osm_data()
osm_vertices = osm_graph.vertices()

# AUX FUNCTION FOR DEBUGGING
def print_graph(graph):
    """
    Prints the vertices and edges of the graph.
    """
    print("Vertices:")
    for vertex in graph.vertices():
        print(f"Vertex ID: {int(vertex)}, lon: {graph.vertex_properties['lon'][vertex]}, lat: {graph.vertex_properties['lat'][vertex]}")

    print("\nEdges:")
    for edge in graph.edges():
        source = int(edge.source())
        target = int(edge.target())
        print(f"Edge: {source} -> {target}")

# AUX FUNCTIONS TO FIND NODES
def find_node_by_coordinates(graph, lon, lat):
    """
    Finds a node in the graph based on its coordinates (lon, lat).

    Parameters:
        graph (graph): the graph containing the node coordinates.
        lon (float): the longitude of the node.
        lat (float): the latitude of the node.

    Returns:
        vertex: the vertex in the graph with the specified coordinates, or None if not found.
    """
    for vertex in graph.vertices():
        if graph.vertex_properties["lon"][vertex] == lon and graph.vertex_properties["lat"][vertex] == lat:
            return vertex
    return None

def find_node_by_id(graph, node_id):
    """
    Finds a node in the graph based on its id.

    Parameters:
        graph (graph): the graph containing the node coordinates.
        node_id (long): the id of the node.

    Returns:
        vertex: the vertex in the graph with the specified id, or None if not found.
    """
    for vertex in graph.vertices():
        if graph.vertex_properties["node_id"][vertex] == node_id:
            return vertex
    return None

def find_nearest_node(graph, latitude, longitude):
    """
    Finds the nearest node in the graph to a given set of coordinates.

    Parameters:
        graph (graph): the graph containing the nodes.
        latitude (float): the latitude of the coordinates.
        longitude (float): the longitude of the coordinates.

    Returns:
        vertex: the vertex in the graph closest to the given coordinates.
    """
    query_point = np.array([longitude, latitude])

    # Obtains vertex properties: 'lon' and 'lat'
    lon_prop = graph.vertex_properties['lon']
    lat_prop = graph.vertex_properties['lat']

    # Calculates the euclidean distances between the node's coordinates and the consulted address's coordinates
    distances = np.linalg.norm(np.vstack((lon_prop.a, lat_prop.a)).T - query_point, axis=1)

    # Finds the nearest node's index
    nearest_node_index = np.argmin(distances)
    nearest_node = graph.vertex(nearest_node_index)

    return nearest_node


def make_undirected(graph):
    """
    Given a directed graph, returns an undirected version of the graph.

    Parameters:
    graph (Graph): A directed graph. In this specific case, the osm graph.

    Returns:
    Graph: An undirected version of the graph.
    """
    undirected_graph = Graph(directed=False)
    vprop_map = graph.new_vertex_property("object")
    
    # Create vertex properties for lon and lat
    lon_prop = undirected_graph.new_vertex_property("float")
    lat_prop = undirected_graph.new_vertex_property("float")
    node_id_prop = undirected_graph.new_vertex_property("long")
    graph_id_prop = undirected_graph.new_vertex_property("long")
    
    # Create edge properties
    u_prop = undirected_graph.new_edge_property("long")
    v_prop = undirected_graph.new_edge_property("long")
    length_prop = undirected_graph.new_edge_property("double")
    weight_prop = undirected_graph.new_edge_property("double")
    
    undirected_vertex_map = {}
    
    for v in graph.vertices():
        new_v = undirected_graph.add_vertex()
        vprop_map[new_v] = v
        lon = graph.vertex_properties["lon"][v]
        lat = graph.vertex_properties["lat"][v]
        node_id = graph.vertex_properties["node_id"][v]
        graph_id = graph.vertex_properties["graph_id"][v]
        
        undirected_vertex_map[node_id] = new_v
        #print("NODO {} EN GRAFO {}".format(node_id, graph_id))
        
        # Assigning node properties
        lon_prop[new_v] = lon
        lat_prop[new_v] = lat
        node_id_prop[new_v] = node_id
        graph_id_prop[new_v] = graph_id
    
    # Assign the properties to the graph
    undirected_graph.vertex_properties["lon"] = lon_prop
    undirected_graph.vertex_properties["lat"] = lat_prop
    undirected_graph.vertex_properties["node_id"] = node_id_prop
    undirected_graph.vertex_properties["graph_id"] = graph_id_prop

    
    for e in graph.edges():
        source, target = e.source(), e.target()
        source_node = graph.edge_properties["u"][e]
        target_node = graph.edge_properties["v"][e]
        lgt = graph.edge_properties["length"][e]
        wt = graph.edge_properties["weight"][e]
        
        if lgt < 2 or source_node == "" or target_node == "":
            continue # Skip edges with empty or missing nodes
            
        if source_node not in undirected_vertex_map or target_node not in undirected_vertex_map:
            print(f"Skipping edge with missing nodes: {source_node} -> {target_node}")
            continue  # Skip edges with missing nodes

        source_vertex = undirected_vertex_map[source_node]
        target_vertex = undirected_vertex_map[target_node]

        if not undirected_graph.vertex(source_vertex) or not undirected_graph.vertex(target_vertex):
            print(f"Skipping edge with non-existent vertices: {source_vertex} -> {target_vertex}")
            continue  # Skip edges with non-existent vertices
            
        e = undirected_graph.add_edge(source_vertex, target_vertex)
        u_prop[e] = source_node
        v_prop[e] = target_node
        length_prop[e] = lgt
        weight_prop[e] = wt
    
    undirected_graph.edge_properties["u"] = u_prop
    undirected_graph.edge_properties["v"] = v_prop
    undirected_graph.edge_properties["length"] = length_prop
    undirected_graph.edge_properties["weight"] = weight_prop
        
    return undirected_graph

# Converts the graph to make an undirected one
undirected_graph = make_undirected(osm_graph)

# Finds the given address in the OSM graph
def address_locator(graph, loc):
    """
    Finds the given address in the OSM graph.

    Parameters:
    graph (Graph): The OSM graph.
    loc (str): The address to be located.

    Returns:
    int: The ID of the nearest vertex in the graph.

    Raises:
    GeocoderServiceError: If there is an error with the geocoding service.

    """
    geolocator = Nominatim(user_agent="ayatori")
    while True:
        try:
            location = geolocator.geocode(loc)
            break
        except GeocoderServiceError:
            i = 0
            if i < 15:
                print("Geocoding service error. Retrying in 5 seconds...")
                tm.sleep(5)
                i+=1
            else:
                msg = "Error: Too many retries. Geocoding service may be down. Please try again later."
                print(msg)
                return
    if location is not None:
        long, lati = location.longitude, location.latitude
        nearest = find_nearest_node(graph,lati,long)
        near_lon, near_lat = graph.vertex_properties["lon"][nearest], graph.vertex_properties["lat"][nearest]
        near_location = geolocator.reverse((near_lat,near_lon))
        near_id = graph.vertex_properties["node_id"][nearest]
        graph_id = graph.vertex_properties["graph_id"][nearest]
        return nearest
    msg = "Error: Address couldn't be found."
    print(msg)

end = tm.time()
exec_time = round((end-start) / 60,3)
print("OSM INFO IS READY. EXECUTION TIME: {} MINUTES".format(exec_time)) 

GETTING OSM INFO
Downloaded Protobuf data 'Santiago.osm.pbf' (19.83 MB) to:
'/home/lysorek/aves/data/external/OSM/Santiago.osm.pbf'
GETTING OSM NODES...
DONE
GETTING OSM EDGES...
OSM DATA HAS BEEN SUCCESSFULLY RECEIVED
OSM INFO IS READY. EXECUTION TIME: 3.53 MINUTES


In [5]:
# THIS IS FOR TESTING IF THE NOMINATIM SERVICE IS UP
a = address_locator(undirected_graph, "aasdas")

Error: Address couldn't be found.


In [6]:
start = tm.time()
print("GETTING GTFS INFO")

## GTFS ##

def get_gtfs_data():
    """
    Reads the GTFS data from a file and creates a directed graph with its info, using the 'pygtfs' library. This gives
    the transit feed data of Santiago's public transport, including "Red Metropolitana de Movilidad" (previously known
    as Transantiago), "Metro de Santiago", "EFE Trenes de Chile", and "Buses de Acercamiento Aeropuerto".
    
    Returns:
        graphs: GTFS data converted to a dictionary of graphs, one per route.
        route_stops: Dictionary containing the stops for each route.
        special_dates: List of special calendar dates.
    """
    # Create a new schedule object using a GTFS file
    sched = pygtfs.Schedule(":memory:")    
    pygtfs.append_feed(sched, "gtfs.zip") # This takes around 2 minutes (01:51.44)
    
    # Get special calendar dates
    special_dates = []
    for cal_date in sched.service_exceptions: # Calendar_dates is renamed in pygtfs
        special_dates.append(cal_date.date.strftime("%d/%m/%Y"))

    # Create a graph per route
    graphs = {}
    stop_id_map = {}  # To assign unique ids to every stop
    stop_coords = {}
    route_stops = {}
    for route in sched.routes:
        graph = Graph(directed=True)
        stop_ids = set()
        trips = [trip for trip in sched.trips if trip.route_id == route.route_id]

        weight_prop = graph.new_edge_property("int")  # Propiedad para almacenar los pesos de las aristas

        for trip in trips:
            stop_times = trip.stop_times

            # Get the orientation of the trip
            orientation = trip.trip_id.split("-")[1]

            for i in range(len(stop_times)):
                stop_id = stop_times[i].stop_id
                sequence = stop_times[i].stop_sequence 

                if stop_id not in stop_id_map:
                    vertex = graph.add_vertex()  # Añadir un vértice vacío
                    stop_id_map[stop_id] = vertex  # Asignar el vértice al identificador de parada
                else:
                    vertex = stop_id_map[stop_id]  # Obtener el vértice existente

                stop_ids.add(vertex)

                if i < len(stop_times) - 1:
                    next_stop_id = stop_times[i + 1].stop_id

                    if next_stop_id not in stop_id_map:
                        next_vertex = graph.add_vertex()  # Añadir un vértice vacío para la siguiente parada
                        stop_id_map[next_stop_id] = next_vertex  # Asignar el vértice al identificador de parada
                    else:
                        next_vertex = stop_id_map[next_stop_id]  # Obtener el vértice existente para la siguiente parada

                    e = graph.add_edge(vertex, next_vertex)  # Añadir una arista entre las paradas
                    weight_prop[e] = 1  # Asignar peso 1 a la arista
                    
                    # Store the coordinates of each stop for this route
                    if route.route_id not in stop_coords:
                        stop_coords[route.route_id] = {}
                    if stop_id not in stop_coords[route.route_id]:
                        stop = sched.stops_by_id(stop_id)[0]
                        stop_coords[route.route_id][stop_id] = (stop.stop_lon, stop.stop_lat)
                        
                        # Store the sequence of each stop for this route
                        if route.route_id not in route_stops:
                            route_stops[route.route_id] = {}
                        route_stops[route.route_id][stop_id] = {
                            "route_id": route.route_id,
                            "stop_id": stop_id,
                            "coordinates": stop_coords[route.route_id][stop_id],
                            "orientation": "round" if orientation == "I" else "return",
                            "sequence": sequence,
                            "arrival_times": []
                        }
                        
                # Get the arrival time for the current stop
                arrival_time = (datetime.min + stop_times[i].arrival_time).time()

                # Check if the stop ID is already in the dictionary
                if stop_id in route_stops[route.route_id]:
                    # If the stop ID is already in the dictionary, append the arrival time
                    route_stops[route.route_id][stop_id]["arrival_times"].append(arrival_time)

        graphs[route.route_id] = graph
        # Group the stops by direction to get the stops visited on the round trip and the return trip
        stops_by_direction = {"round_trip": [], "return_trip": []}
        for trip in trips:
            stop_times = trip.stop_times
            stops = [stop_times[i].stop_id for i in range(len(stop_times))]
            
            # Determine the direction of the trip
            if trip.direction_id == 0:
                stops_by_direction["round_trip"].extend(stops)
            else:
                stops_by_direction["return_trip"].extend(stops)


        # Get the unique stops visited on the round trip and the return trip
        round_trip_stops = set(stops_by_direction["round_trip"])
        return_trip_stops = set(stops_by_direction["return_trip"])
        
        for stop_id in round_trip_stops:
            if stop_id in stop_coords[route.route_id]:
                if stop_id in route_stops[route.route_id]:
                    route_stops[route.route_id][stop_id]["orientation"] = "round"
                else:
                    route_stops[route.route_id][stop_id] = {
                        "route_id": route.route_id,
                        "stop_id": stop_id,
                        "coordinates": stop_coords[route.route_id][stop_id],
                        "orientation": "round",
                        "sequence": sequence,
                        "arrival_times": []
                    }
        for stop_id in return_trip_stops:
            if stop_id in stop_coords[route.route_id]:
                if stop_id in route_stops[route.route_id]:
                    route_stops[route.route_id][stop_id]["orientation"] = "return"
                else:
                    route_stops[route.route_id][stop_id] = {
                        "route_id": route.route_id,
                        "stop_id": stop_id,
                        "coordinates": stop_coords[route.route_id][stop_id],
                        "orientation": "return",
                        "sequence": sequence,
                        "arrival_times": []
                    }

    print("DONE")
    print("STORING ROUTE GRAPHS...")

    # Store graphs into a file
    for route_id, graph in graphs.items():
        weight_prop = graph.new_edge_property("int")  # Crear una nueva propiedad de peso de arista

        for e in graph.edges():  # Iterar sobre las aristas del grafo
            weight_prop[e] = 1  # Asignar el peso 1 a cada arista

        graph.edge_properties["weight"] = weight_prop  # Asignar la propiedad de peso al grafo
        
        data_dir = "gtfs_routes"
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)
        graph.save(f"{data_dir}/{route_id}.gt")
    
    print("GTFS DATA RECEIVED SUCCESSFULLY")
    return graphs, route_stops, special_dates

# GTFS Graph
gtfs_graph, route_stops, special_dates = get_gtfs_data()

end = tm.time()
exec_time = round((end-start) / 60,3)
print("GTFS INFO IS READY. EXECUTION TIME: {} MINUTES".format(exec_time)) 

GETTING GTFS INFO
Loading GTFS data for <class 'pygtfs.gtfs_entities.Agency'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Stop'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Transfer'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Route'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Fare'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.FareRule'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.ShapePoint'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Service'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.ServiceException'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Trip'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Frequency'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.StopTime'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.FeedInfo'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Translation'>:
4 records read for <class 'pygtfs.gtfs_entities.Agency'>.
..11663 records read for <class 'pygtfs.gtfs_e

In [7]:
# MAPPING FUNCTIONS

In [32]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points on the earth (specified in decimal degrees).
    
    Parameters:
    lon1 (float): Longitude of the first point in decimal degrees.
    lat1 (float): Latitude of the first point in decimal degrees.
    lon2 (float): Longitude of the second point in decimal degrees.
    lat2 (float): Latitude of the second point in decimal degrees.
    
    Returns:
    float: The distance between the two points in kilometers.
    """
    R = 6372.8  # Earth radius in kilometers
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    a = sin(dLat / 2)**2 + cos(lat1) * cos(lat2) * sin(dLon / 2)**2
    c = 2 * asin(sqrt(a))
    return R * c

map_colors= ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 
         'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue',
         'lightgreen', 'gray', 'black', 'lightgray']

def map_route_stops(route_list, color_list, stops_flag):
    """
    Create a map showing the stops visited on the round trip for the specified routes.
    
    Parameters:
    route_list (list): A list of route IDs.
    color_list (list): A list of colors for each route.
    stops_flag (bool): A flag indicating whether to display the stops on the map.
    
    Returns:
    folium.Map: A map object showing the stops and routes.
    """
    # Map the stops visited on the round trip
    map = folium.Map(location=[-33.45, -70.65], zoom_start=12)
    
    color_id = 0
    for route_id in route_list:
        # Get the stops for the specified route
        stops = route_stops.get(route_id, {})

        # Filter the stops that are visited on the round trip
        round_trip_stops = [stop_info for stop_info in stops.values() if stop_info["orientation"] == "round"]

        # Sort the stops by their sequence number in the trip
        round_trip_stops.sort(key=lambda stop: stop["sequence"])
        for stop in round_trip_stops:
            print(stop['sequence'])

        if stops_flag:
            for stop_info in round_trip_stops:
                folium.Marker(location=[stop_info["coordinates"][1], stop_info["coordinates"][0]], popup=stop_info["stop_id"],
                               icon=folium.Icon(color='lightgray', icon='minus')).add_to(map)

        folium.PolyLine(locations=[[stop_info["coordinates"][1], stop_info["coordinates"][0]] for stop_info in round_trip_stops],
                        color=map_colors[color_id], weight=2).add_to(map)
        
        color_id+=1

    return map

def get_stop_coords(route_stops, stop_id):
    """
    Given a dictionary of route stops and a stop ID, returns the coordinates of the stop with the given ID.
    If the stop ID is not found, returns None.

    Parameters:
    route_stops (dict): A dictionary of route stops.
    stop_id (int): The ID of the stop to get the coordinates for.

    Returns:
    tuple: A tuple of two floats representing the longitude and latitude of the stop with the given ID.
    None: If the stop ID is not found.
    """
    for route_id, stops in route_stops.items():
        for stop_info in stops.values():
            if stop_info["stop_id"] == stop_id:
                return stop_info["coordinates"]
    return None

def get_near_stop_ids(route_stops, coords, margin):
    """
    Given a dictionary of route stops, a tuple of coordinates, and a margin, returns a list of stop IDs
    that are within the specified margin of the given coordinates, along with their orientations.

    Parameters:
    route_stops (dict): A dictionary of route stops.
    coords (tuple): A tuple of two floats representing the longitude and latitude of the coordinates to search around.
    margin (float): The maximum distance (in kilometers) from the given coordinates to include stops in the result.

    Returns:
    tuple: A tuple of two lists. The first list contains the stop IDs that are within the specified margin of the given coordinates.
    The second list contains tuples of stop IDs and their orientations.
    """
    stop_ids = []
    orientations = []
    for route_id, stops in route_stops.items():
        for stop_info in stops.values():
            stop_coords = stop_info["coordinates"]
            distance = haversine(coords[1], coords[0], stop_coords[1], stop_coords[0])
            if distance <= margin:
                orientation = stop_info["orientation"]
                stop_id = stop_info["stop_id"]
                if stop_id not in stop_ids:
                    stop_ids.append(stop_id)
                    orientations.append((stop_id, orientation))
    return stop_ids, orientations

def find_nearest_stops(address, margin):
    """
    Given an address and a margin, returns a list of the nearest stop IDs and their orientations.

    Parameters:
    address (str): The address to search around.
    margin (float): The maximum distance (in kilometers) from the given address to include stops in the result.

    Returns:
    tuple: A tuple of two lists. The first list contains the stop IDs that are within the specified margin of the given address.
    The second list contains tuples of stop IDs and their orientations.
    """
    v = address_locator(undirected_graph, str(address))
    v_lon = undirected_graph.vertex_properties['lon'][v]
    v_lat = undirected_graph.vertex_properties['lat'][v]
    v_coords = (v_lon, v_lat)
    nearest_stops, orientations = get_near_stop_ids(route_stops, v_coords, margin)
    return nearest_stops, orientations


def get_route_stop_ids(route_stops, route_id):
    """
    Given a dictionary of routes and stops, and a route ID, returns a list of stop IDs for the stops on the given route.

    Parameters:
    route_stops (dict): A dictionary of route stops.
    route_id (int): The ID of the route to get the stops for.

    Returns:
    list: A list of stop IDs for the stops on the given route.
    """
    stops = route_stops.get(route_id, {})
    return stops.keys()

def route_stop_matcher(route_stops, route_id, stop_id):
    """
    Given a dictionary of routes and stops, a route ID, and a stop ID, returns True if the stop ID is on the given route,
    and False otherwise.

    Parameters:
    route_stops (dict): A dictionary of route stops.
    route_id (int): The ID of the route to check.
    stop_id (int): The ID of the stop to check.

    Returns:
    bool: True if the stop ID is on the given route, False otherwise.
    """
    stop_list = get_route_stop_ids(route_stops, route_id)
    return (stop_id in stop_list)

def is_route_near_coordinates(route_stops, route_id, coordinates, margin):
    """
    Given a dictionary of routes and stops, a route ID, a tuple of coordinates, and a margin, returns True if the route
    has a stop within the specified margin of the given coordinates, and False otherwise.

    Parameters:
    route_stops (dict): A dictionary of route stops.
    route_id (int): The ID of the route to check.
    coordinates (tuple): A tuple of two floats representing the longitude and latitude of the coordinates to search around.
    margin (float): The maximum distance (in kilometers) from the given coordinates to include stops in the result.

    Returns:
    bool: True if the route has a stop within the specified margin of the given coordinates, False otherwise.
    """
    for stop_info in route_stops[route_id].values():
        #print(stop_info["stop_id"])
        stop_coords = stop_info["coordinates"]
        distance = haversine(coordinates[1], coordinates[0], stop_coords[1], stop_coords[0])
        if distance <= margin:
            return route_id
    return False


def get_bus_orientation(route_id, stop_id):
    """
    Checks and confirms the bus orientation, while visiting a stop, in the GTFS data files.

    Parameters:
    route_id (str): The route or service's ID to check.
    stop_id (str): The visited stop ID.

    Returns:
    str or list: The bus orientation(s) associated with the route_id and stop_id. None if nothing is found.
    """
    stop_times = pd.read_csv("stop_times.txt")
    filtered_stop_times = stop_times[(stop_times["trip_id"].str.startswith(route_id)) & (stop_times["stop_id"] == stop_id)]
    
    orientations = []
    for trip_id in filtered_stop_times["trip_id"]:
        orientation = trip_id.split("-")[1]
        if orientation == "I" and "round" not in orientations:
            orientations.append("round")
        elif orientation == "R" and "return" not in orientations:
            orientations.append("return")
    
    if len(orientations) == 0:
        return None
    elif len(set(orientations)) == 1:
        return orientations[0]
    else:
        return orientations

def connection_finder(route_stops, stop_id_1, stop_id_2):
    """
    Finds all routes that have stops at both given stop IDs.

    Parameters:
    route_stops (dict): A dictionary containing information about stops on a route.
    stop_id_1 (str): The ID of the first stop to check.
    stop_id_2 (str): The ID of the second stop to check.

    Returns:
    list: A list of route IDs that have stops at both given stop IDs.
    """
    connected_routes = []
    for route_id, stops in route_stops.items():
        stop_ids = [stop_info["stop_id"] for stop_info in stops.values()]

        if stop_id_1 in stop_ids and stop_id_2 in stop_ids:
            connected_routes.append(route_id)
    return connected_routes

def get_routes_at_stop(route_stops, stop_id):
    """
    Finds all routes that have a stop at the given stop ID.

    Parameters:
    route_stops (dict): A dictionary containing information about stops on a route.
    stop_id (str): The ID of the stop to check.

    Returns:
    list: A list of route IDs that have a stop at the given stop ID.
    """
    routes = [route_id for route_id in route_stops.keys() if stop_id in get_route_stop_ids(route_stops, route_id) and connection_finder(route_stops, stop_id, stop_id)]
    return routes

def is_24_hour_service(route_id):
    """
    Determines if the given route has a 24-hour service.

    Parameters:
    route_id (str): A string representing the ID of the route.

    Returns:
    bool: True if the route has a 24-hour service, False otherwise.
    """
    # Read the frequencies for the route
    frequencies = pd.read_csv("frequencies.txt")
    route_str = str(route_id) + "-"
    route_frequencies = frequencies[frequencies["trip_id"].str.startswith(route_str)]

    # Check if any frequency has a start time of "00:00:00" and an end time of "24:00:00"
    has_start_time = False
    has_end_time = False
    for _, row in route_frequencies.iterrows():
        start_time = row["start_time"]
        end_time = row["end_time"]
        if start_time == "00:00:00":
            has_start_time = True
        if end_time == "24:00:00":
            has_end_time = True

    return has_start_time and has_end_time

def check_night_routes(valid_services, is_nighttime):
    """
    Filters the given list of route IDs to only include night routes if is_nighttime is True.

    Parameters:
    valid_services (list): A list of route IDs to filter.
    is_nighttime (bool): True if it is nighttime, False otherwise.

    Returns:
    list: A list of route IDs that are night routes if is_nighttime is True, or all route IDs otherwise.
    """
    if is_nighttime:
        #nighttime_routes = [route_id for route_id in valid_services if route_id.endswith("N")]
        nighttime_routes = [route_id for route_id in valid_services if route_id.endswith("N") or is_24_hour_service(route_id)]
        if nighttime_routes:
            return nighttime_routes
        else:
            return None
    else:
        daytime_routes = [route_id for route_id in valid_services if not route_id.endswith("N")]
        if daytime_routes:
            return daytime_routes
        else:
            return None

def is_nighttime(source_hour):
    """
    Determines if the given hour is during the nighttime.

    Parameters:
    source_hour (datetime.time): The hour to check.

    Returns:
    bool: True if the hour is during the nighttime, False otherwise.
    """
    start_time = time(0, 0, 0)
    end_time = time(5, 30, 0)
    #source_time = datetime.strptime(source_hour, "%H:%M:%S").time()
    if start_time <= source_hour <= end_time:
        return True
    else:
        return False
    
    
def check_express_routes(valid_services, is_rush_hour):
    """
    Filters the given list of route IDs to only include express routes if is_rush_hour is True.

    Parameters:
    valid_services (list): A list of route IDs to filter.
    is_rush_hour (bool): True if it is rush hour, False otherwise.

    Returns:
    list: A list of route IDs that are express routes if is_rush_hour is True, or all route IDs otherwise.
    """
    if is_rush_hour:
        return valid_services
    else:
        regular_hour_routes = [route_id for route_id in valid_services if not route_id.endswith("e")]
        return regular_hour_routes
    
def is_rush_hour(source_hour):
    """
    Determines if the given hour is during rush hour.

    Parameters:
    source_hour (datetime.time): The hour to check.

    Returns:
    bool: True if the hour is during rush hour, False otherwise.
    """
    am_start_time = time(5, 30, 0)
    am_end_time = time(9, 0, 0)
    pm_start_time = time(17, 30, 0)
    pm_end_time = time(21, 0, 0)
    #source_time = datetime.strptime(source_hour, "%H:%M:%S").time()
    if am_start_time <= source_hour <= am_end_time or pm_start_time <= source_hour <= pm_end_time:
        return True
    else:
        return False
    
def is_holiday(date_string):
    """
    Checks if a given date is a holiday.

    Parameters:
    date_string (str): A string representing the date in the format "dd/mm/yyyy".

    Returns:
    bool: True if the date is a holiday, False otherwise.
    """
    # Local holidays
    if date_string in special_dates:
        return True
    date_obj = datetime.strptime(date_string, "%d/%m/%Y")

    # Weekend days
    day_of_week = date_obj.weekday()
    if day_of_week == 5 or day_of_week == 6:
        return True
    return False


def get_trip_day_suffix(date):
    """
    Based on the given date, gets the corresponding trip day suffix for the trip IDs.
    
    Parameters:
    date (date): The date to be checked.
    
    Returns
    str: A string with the trip day suffix.
    """
    # Get the day of the week
    date_object = datetime.strptime(date, "%d/%m/%Y")
    day_of_week = date_object.weekday()

    # Determine the trip ID suffix based on the day of the week
    if day_of_week < 5:
        trip_day_suffix = "L"
    elif day_of_week == 5:
        trip_day_suffix = "S"
    else:
        trip_day_suffix = "D"

    return trip_day_suffix

def get_arrival_times(route_id, stop_id, source_date):
    """
    Returns the arrival times for a given route and stop.

    Parameters:
    route_id (str): A string representing the ID of the route.
    stop_id (str): A string representing the ID of the stop.

    Returns:
    tuple: A tuple containing a string representing the bus orientation ("round" or "return") and a list of datetime objects representing the arrival times.
    """
    # Read the frequencies.txt file
    frequencies = pd.read_csv("frequencies.txt")

    # Filter the frequencies for the given route ID
    route_frequencies = frequencies[frequencies["trip_id"].str.startswith(route_id)]
    
    # Get the day suffix
    day_suffix = get_trip_day_suffix(source_date)

    # Get the arrival times for the stop for each trip
    stop_route_times = []
    bus_orientation = ""
    for _, row in route_frequencies.iterrows():
        start_time = pd.Timestamp(row["start_time"])
        if row["end_time"] == "24:00:00":
            end_time = pd.Timestamp("23:59:59")
        else:
            end_time = pd.Timestamp(row["end_time"])
        headway_secs = row["headway_secs"]
        round_trip_id = f"{route_id}-I-{day_suffix}"
        return_trip_id = f"{route_id}-R-{day_suffix}"
        round_stop_times = pd.read_csv("stop_times.txt").query(f"trip_id.str.startswith('{round_trip_id}') and stop_id == '{stop_id}'")
        return_stop_times = pd.read_csv("stop_times.txt").query(f"trip_id.str.startswith('{return_trip_id}') and stop_id == '{stop_id}'")
        if len(round_stop_times) == 0 and len(return_stop_times) == 0:
            return
        elif len(round_stop_times) > 0:
            bus_orientation = "round"
            stop_time = pd.Timestamp(round_stop_times.iloc[0]["arrival_time"])
        elif len(return_stop_times) > 0:
            bus_orientation = "return"
            stop_time = pd.Timestamp(return_stop_times.iloc[0]["arrival_time"])
        for freq_time in pd.date_range(start_time, end_time, freq=f"{headway_secs}s"):
            freq_time_str = freq_time.strftime("%H:%M:%S")
            freq_time = datetime.strptime(freq_time_str, "%H:%M:%S")
            stop_route_time = datetime.combine(datetime.min, stop_time.time()) + timedelta(seconds=(freq_time - datetime.min).seconds)
            if stop_route_time not in stop_route_times:
                stop_route_times.append(stop_route_time)
            stop_time += pd.Timedelta(seconds=headway_secs)

    return bus_orientation, stop_route_times

def get_time_until_next_bus(arrival_times, source_hour, source_date):
    """
    Returns the time until the next three buses.

    Parameters:
    arrival_times (list): A list of datetime objects representing the arrival times of the buses.
    source_hour (datetime.time): The source hour to compare with the arrival times.
    source_date (datetime.date): The source date to check if there are buses remaining.

    Returns:
    list: A list of tuples representing the time until the next three buses in minutes and seconds.
    """
    arrival_times_remaining = []
    for a_time in arrival_times:
        if a_time.time() >= source_hour:
            arrival_times_remaining.append(a_time)
    #arrival_times_remaining = [time for time in arrival_times if time.time() >= source_hour]
    if len(arrival_times_remaining) == 0:
        return None
    else:
        # Sort the remaining arrival times in ascending order
        arrival_times_remaining.sort()

        # Get the datetime objects for the next three buses
        next_buses = []
        for i in range(min(3, len(arrival_times_remaining))):
            next_arrival_time = arrival_times_remaining[i]
            next_bus = datetime.combine(next_arrival_time.date(), next_arrival_time.time())
            next_buses.append(next_bus)
        
        if next_buses is None:
            print("No buses remaining for the specified date.")
        else:
            # Calculate the time until the next three buses
            time_until_next_buses = []
            for next_bus in next_buses:
                time_until_next_bus = (next_bus - datetime.combine(next_bus.date(), source_hour)).total_seconds()
                minutes, seconds = divmod(time_until_next_bus, 60)
                time_until_next_buses.append((int(minutes), int(seconds)))
            
            return time_until_next_buses


def timedelta_to_hhmm(td):
    """
    Converts a timedelta object to a string in HHMM format.
    
    Parameters:
    td (timedelta): The timedelta object to be converted.
    
    Returns:
    str: A formated string with the time.
    """
    total_seconds = int(td.total_seconds())
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    return f"{hours:02d}:{minutes:02d}"
    
def timedelta_separator(td):
    """
    Separates a timedelta object into minutes and seconds.

    Parameters:
    td (timedelta): A timedelta object representing a duration of time.

    Returns:
    tuple: A tuple containing the number of minutes and seconds in the timedelta object. The minutes and seconds are both integers.
    """
    total_seconds = td.total_seconds()
    minutes = int(total_seconds // 60)
    seconds = int(total_seconds % 60)
    return minutes, seconds
    
    
def get_travel_time(trip_id, stop_ids):
    """
    Returns the travel time between two stops for a given trip.

    Parameters:
    trip_id (str): A string representing the ID of the trip.
    stop_ids (list): A list of two strings representing the IDs of the stops.

    Returns:
    timedelta: A timedelta object representing the travel time.
    """
    stop_times = pd.read_csv("stop_times.txt").query(f"trip_id.str.startswith('{trip_id}') and stop_id in {stop_ids}")
    if len(stop_times) < 2:
        return None
    arrival_times = [datetime.strptime(arrival_time, "%H:%M:%S") for arrival_time in stop_times["arrival_time"]]
    travel_time = arrival_times[1] - arrival_times[0]
    return travel_time

def get_trip_sequence(route_stops, route_id, stop_id):
    """
    Given a dictionary of routes and stops, a route ID and a stop ID, gets the trip sequence number corresponding to the stop.
    
    Parameters:
    route_stops (dict): Dictionary of transport data (routes and stops).
    route_id (str): The route or service's ID.
    stop_id (str): The stop's ID.
    
    Returns:
    str: A string representing the sequence number.
    """
    seq = route_stops[route_id][stop_id]["sequence"]
    return seq

def walking_travel_time(stop_coords, location_coords, speed):
    """
    Calculates the walking travel time between a location and a stop, given a speed value.
    
    Parameters:
    stop_coords (tuple): A tuple with the stop's coordinates.
    location_coords (tuple):  A tuple with the location's coordinates.
    speed (float): The walking speed value.
    
    Returns.
    float: The time (in seconds) that represents the travel time.
    """
    distance = haversine(stop_coords[0], stop_coords[1], location_coords[0], location_coords[1])
    time = round((distance / speed) * 3600,2)
    return time

def parse_metro_stations(stops_file):
    """
    Parses the Metro Stations data, creating a dictionary with their names.
    
    Parameters:
    stops_file (File): The GTFS file with the stop data (stops.txt).
    
    Returns:
    dict: A dictionary with the names of the stations.
    """
    subway_stops = {}
    with open(stops_file, 'r') as f:
        for line in f:
            stop_id, _, stop_name, _, _, _, _ = line.strip().split(',')
            if stop_id.isdigit():
                subway_stops[stop_id] = stop_name
    return subway_stops

def is_metro_station(stop_id, route_dict):
    """
    Checks if a stop is a Metro station.
    
    Parameters:
    stop_id (str): The stop's ID to be checked.
    route_dict (dict): The dictionary with the Metro stations names.
    
    Returns:
    str or None: A string with the stop ID if the stop is a Metro station, or None if it isn't.
    """
    try:
        route_num = int(stop_id)
        return route_dict[stop_id]
    except ValueError:
        return None

# Define the function to set the optimal zoom level for the map
def fit_bounds(points, m):
    """
    Fits the map bounds to a given set of points.

    Parameters:
    points (list): A list of points in the format [(lat1, lon1), (lat2, lon2), ...].
    m (folium.Map): A folium map object.
    """
    df = pd.DataFrame(points).rename(columns={0:'Lat', 1:'Lon'})[['Lat', 'Lon']]
    sw = df[['Lat', 'Lon']].min().values.tolist()
    ne = df[['Lat', 'Lon']].max().values.tolist()
    m.fit_bounds([sw, ne])

In [33]:
# ALGORITHM SCRIPT

In [36]:
def connection_scan(source_address, target_address, departure_time, departure_date, margin):
    """
    The Connection Scan algorithm is applied to search for travel routes from the source to the destination,
    given a departure time and date. By default, the algorithm uses the current date and time of the system.
    However, you can specify a different date or time if needed.
    The margin value let's the user determine the range on which a stop is considered as "near" to the source
    or target addresses.

    Parameters:
    source_address (string): the source address of the travel.
    target_address (string): the destination address of the travel.
    departure_time (time): the time at which the travel should start.
    departure_date (date): the date on which the travel should be done.
    margin (float): margin of distance between the nodes and the valid stops.

    Returns:
    folium.Map: the map of the best travel route. It returns None if no routes are found.
    """
    # Getting the nodes corresponding to the addresses
    graph = undirected_graph
    source_node = address_locator(graph, source_address)
    target_node = address_locator(graph, target_address)

    if source_node is not None and target_node is not None:
        # Convert source and target node IDs to integers
        source_node_graph_id = graph.vertex_properties["graph_id"][source_node]
        target_node_graph_id = graph.vertex_properties["graph_id"][target_node]

        print("Both addresses have been found.")
        print("Processing...")

        selected_path_nodes = [source_node_graph_id, target_node_graph_id]
        selected_path = []
        for node in selected_path_nodes:
            # Getting the coordinates
            lat, lon = graph.vertex_properties["lat"][node], graph.vertex_properties["lon"][node]
            selected_path.append((lat, lon))

        geolocator = Nominatim(user_agent="ayatori")
        # Coordinates
        source_lat = selected_path[0][0]
        source_lon = selected_path[0][1]
        target_lat = selected_path[-1][0]
        target_lon = selected_path[-1][1]
        # Reversing the geocoding to get the full info on the addresses
        source = geolocator.reverse((source_lat,source_lon))
        target = geolocator.reverse((target_lat,target_lon))

        # Create a map that shows the correct public transport services to take from the source to the target
        m = folium.Map(location=[selected_path[0][0], selected_path[0][1]], zoom_start=13)

        # Add markers for the source and target points
        folium.Marker(location=[selected_path[0][0], selected_path[0][1]], popup="Origen: {}".format(source), icon=folium.Icon(color='green')).add_to(m)
        folium.Marker(location=[selected_path[-1][0], selected_path[-1][1]], popup="Destino: {}".format(target), icon=folium.Icon(color='red')).add_to(m)

        # Add markers for the nearest stop from the source and target points
        near_source_stops, source_orientations = find_nearest_stops(source, margin)
        near_target_stops, target_orientations = find_nearest_stops(target, margin)

        fixed_orientation = None
        valid_services = set()
        for source_stop_id in near_source_stops:
            for target_stop_id in near_target_stops:
                # Finding services that connects a stop near the source with one near the target
                services = connection_finder(route_stops, source_stop_id, target_stop_id)
                for service in services:
                    # Getting the orientations on which the services visits the stops
                    source_orientation = get_bus_orientation(service, source_stop_id)
                    target_orientation = get_bus_orientation(service, target_stop_id)
                    # Getting the sequence number (the ordinal value for the visited stop)
                    source_sequence = int(get_trip_sequence(route_stops, service, source_stop_id))
                    target_sequence = int(get_trip_sequence(route_stops, service, target_stop_id))
                    if source_sequence > target_sequence:
                        # Travel not valid
                        continue
                    if isinstance(source_orientation, list) and isinstance(target_orientation, list):
                        # If both source and target orientations are lists, check if any of the values match
                        valid_orientation = any(x in target_orientation for x in source_orientation) or any(x in source_orientation for x in target_orientation)
                        if valid_orientation and service not in valid_services:
                            valid_services.add(service)
                            fixed_orientation = [x for x in source_orientation if x in target_orientation][0] if [x for x in source_orientation if x in target_orientation] else source_orientation[0]
                    elif source_orientation == target_orientation and service not in valid_services:  # Check if both stops are visited in the same orientation
                        valid_services.add(service)
                        fixed_orientation = target_orientation
                    elif isinstance(source_orientation, list) and target_orientation in source_orientation and service not in valid_services:
                        valid_services.add(service)
                        fixed_orientation = target_orientation
                    elif isinstance(target_orientation, list) and source_orientation in target_orientation and service not in valid_services:
                        valid_services.add(service)
                        fixed_orientation = source_orientation

        if len(valid_services) == 0:
            print("Error: There are no available services right now to go to the desired destination.")
            print("Possible reasons: no routes that have stops near the source and target addresses.")
            print("You can try changing the search margin and try again.")
            return

        # Checking flags for time and date
        nighttime_flag = is_nighttime(departure_time)
        rush_hour_flag = is_rush_hour(departure_time)
        holiday_flag = is_holiday(departure_date)
        
        if holiday_flag:
            # Rush hours modifications (like express routes) doesn't work on holidays
            rush_hour_flag = 0

        # Nighttime check
        daily_time_services = check_night_routes(valid_services, nighttime_flag)

        if daily_time_services is None:
            print("Error: There are no available services right now to go to the desired destination.")
            print("Possible reasons: Source hour is during nighttime.")
            print("Please take into account that nighttime goes between 00:00:00 and 05:30:00.")
            return

        # Rush hour check
        valid_services = check_express_routes(daily_time_services, rush_hour_flag)
        # Sorting the valid services
        valid_services = list(set(valid_services))

        # Filters the stops to get only the valid ones
        valid_source_stops = [stop_id for stop_id in near_source_stops if any(route_id in valid_services for route_id in route_stops.keys() if stop_id in route_stops[route_id])]
        valid_source_stops = list(set(valid_source_stops))
        valid_target_stops = [stop_id for stop_id in near_target_stops if any(route_id in valid_services for route_id in route_stops.keys() if stop_id in route_stops[route_id])]
        valid_target_stops = list(set(valid_target_stops))

        print("")
        print("Routes have been found.")
        print("Calculating the best route and getting the arrival times for the next buses...")

        best_option = None
        best_option_times = None
        initial_source_time = timedelta(hours=departure_time.hour, minutes=departure_time.minute, seconds=departure_time.second)
        source_time = None

        # Set of valid orientations defined by the source
        valid_orientations = set(source_orientations)
        best_option_orientation = None

        # Checks for valid target stops
        valid_target = []
        for target_stop in valid_target_stops:
            target_routes = get_routes_at_stop(route_stops, target_stop)
            valid_target.extend(target_routes)
        valid_target = list(dict.fromkeys(valid_target))

        waiting_time = None
        initial_delta_time = None

        for stop_id in valid_source_stops:
            # Gets the services that visits the stop and filters the valid ones (source)
            routes_at_stop = get_routes_at_stop(route_stops, stop_id)
            valid_stop_services = [stop_id for stop_id in valid_services if stop_id in routes_at_stop]
            for valid_service in valid_stop_services:
                # Gets the arrival times and service orientation for this valid service
                arrival_info = get_arrival_times(valid_service, stop_id, departure_date)
                if arrival_info is not None and arrival_info[0] == fixed_orientation:
                    orientation = arrival_info[0]
                    flag = False # A flag to correct the orientation
                    for target_stop_id in valid_target_stops:
                        flag = False
                        # Gets the services that visits the stop and filters the valid ones (target)
                        target_stop_routes = get_routes_at_stop(route_stops, target_stop_id)
                        if valid_service in target_stop_routes:
                            target_orientation = get_bus_orientation(valid_service, target_stop_id)
                            if orientation not in target_orientation:
                                flag = True
                                continue

                    if flag:
                        continue

                    if valid_service not in valid_target:
                        continue

                    arrival_times = arrival_info[1]

                    # Gets the coordinates for the stop and the source location
                    stop_coords = get_stop_coords(route_stops, stop_id)
                    location_coords = (source_lon, source_lat)

                    # Consider the travel time between the source location and the stop
                    # The average walking speed is between 4 km/h and 6.5 km/h, so we consider it as 5 km/h
                    initial_walking_time = walking_travel_time(stop_coords, location_coords, 5) 
                    this_delta_time = timedelta(seconds=initial_walking_time)

                    initial_time = (datetime.combine(date.today(), departure_time) + this_delta_time).time().strftime("%H:%M:%S")
                    initial_time = datetime.strptime(initial_time, "%H:%M:%S").time()
                    source_time = timedelta(hours=initial_time.hour, minutes=initial_time.minute, seconds=initial_time.second)

                    # Getting the times of the next buses arrival to the stop
                    time_until_next_buses = get_time_until_next_bus(arrival_times, initial_time, departure_date)

                    if not time_until_next_buses:
                        print("Error: There are no available services right now to go to the desired destination.")
                        print("Possible reasons: There are no buses left today. Maybe the source hour is too close to the ending time for the service.")
                        return


                    # Print the time until the next three buses in the desired format
                    for i in range(len(time_until_next_buses)):
                        minutes, seconds = time_until_next_buses[i]
                        waiting_time = timedelta(minutes=minutes, seconds=seconds)
                        arrival_time = source_time + waiting_time
                        time_string = timedelta_to_hhmm(arrival_time)

                        target_orientation = get_bus_orientation(valid_service, target_stop_id)

                        # Update the best option
                        if (best_option is None or (arrival_time < best_option[2])) and orientation == fixed_orientation:
                            best_option = (valid_service, stop_id, arrival_time, waiting_time)
                            best_option_times = time_until_next_buses
                            best_option_orientation = orientation
                            if initial_delta_time is None or this_delta_time < initial_delta_time:
                                initial_delta_time = this_delta_time

        if best_option is None:
            print("Error: There are no available services right now to go to the desired destination.")
            print("Possible reasons: the valid routes are not available at the specified date or starting time.")
            print("Please take into account that some routes have trips only during or after nighttime, which goes between 00:00:00 and 05:30:00")
            return

        arrival_time = None

        source_stop = best_option[1]

        # Parse Metro stations's names
        metro_stations_dict = parse_metro_stations("stops.txt")
        possible_metro_name = is_metro_station(best_option[1], metro_stations_dict)
        if possible_metro_name is not None:
            source_stop = possible_metro_name

        walking_minutes, walking_seconds = timedelta_separator(initial_delta_time)

        print("")
        print("To go from: {}".format(source))
        print("To: {}".format(target))
        best_arrival_time_str = timedelta_to_hhmm(best_option[2])
        print("")
        if possible_metro_name is not None: # Changes the printing to adapt for the use of Metro
            print("The best option is to walk for {} minutes and {} seconds to {} Metro station, and take the line {}.".format(walking_minutes, walking_seconds, source_stop, best_option[0]))
            print("The next train arrives at {}.".format(best_arrival_time_str))
            print("The other two next trains arrives in:")
        else:
            print("The best option is to walk for {} minutes and {} seconds to stop {}, and take the route {}.".format(walking_minutes, walking_seconds, source_stop, best_option[0]))
            print("The next bus arrives at {}.".format(best_arrival_time_str))
            print("The other two next buses arrives in:")

        # Format and prints the times
        for i in range(len(best_option_times)):
            if i == 0:
                continue
            minutes, seconds = best_option_times[i]
            waiting_time = timedelta(minutes=minutes, seconds=seconds)
            arrival_time = initial_source_time + waiting_time
            time_string = timedelta_to_hhmm(arrival_time)
            print(f"{minutes} minutes, {seconds} seconds ({time_string})")


        for stop_id in near_source_stops:
            if stop_id in valid_source_stops:
                # Filters the data for selecting the best source option for its mapping
                stop_coords = get_stop_coords(route_stops, str(stop_id))
                routes_at_stop = get_routes_at_stop(route_stops, stop_id)
                valid_stop_services = [stop_id for stop_id in valid_services if stop_id in routes_at_stop]

                for service in valid_stop_services:
                    if service == best_option[0] and stop_id == best_option[1]:
                        # Maps the best option to take the best option's service
                        folium.Marker(location=[stop_coords[1], stop_coords[0]], 
                              popup="Mejor opción: subirse al recorrido {} en la parada {}.".format(best_option[0], best_option[1]), 
                              icon=folium.Icon(color='cadetblue', icon='plus')).add_to(m)
                        initial_distance = [(selected_path[0][0], selected_path[0][1]),(stop_coords[1], stop_coords[0])]
                        folium.PolyLine(initial_distance,color='black',dash_array='10').add_to(m)

        for stop_id in near_target_stops:
            if stop_id in valid_target_stops:
                # Filters the data for the possible target stops
                stop_coords = get_stop_coords(route_stops, str(stop_id))
                routes_at_stop = get_routes_at_stop(route_stops, stop_id)
                valid_stop_services = [stop_id for stop_id in valid_services if stop_id in routes_at_stop]

        target_orientation = None
        for service in valid_target:
            if service == best_option[0]:
                # Generates the trip id to get the approximated travel time
                if fixed_orientation == "round":
                    trip_id = service + "-I-" + get_trip_day_suffix(departure_date)
                else:
                    trip_id = service + "-R-" + get_trip_day_suffix(departure_date)

                best_travel_time = None
                selected_stop = None 
                for stop_id in valid_target_stops:
                    # Calculates the travel time while taking the service
                    bus_time = get_travel_time(trip_id, [best_option[1], stop_id])
                    target_stop_routes = get_routes_at_stop(route_stops, stop_id)
                    target_orientation = get_bus_orientation(best_option[0], stop_id)
                    if service in target_stop_routes and bus_time > timedelta() and (best_travel_time is None or bus_time < best_travel_time):
                        # Checking the correct orientation
                        if fixed_orientation in target_orientation:
                            # Updates the selected target stop and travel time
                            best_travel_time = bus_time
                            selected_stop = stop_id 

                # Gets the coordinates for the target stop
                selected_stop_coords = get_stop_coords(route_stops, selected_stop)
                # Separates the best travel time for the printing
                minutes, seconds = timedelta_separator(best_travel_time)

                # Gets the sequence number for the source and target stops
                seq_1 = route_stops[best_option[0]][best_option[1]]["sequence"]
                seq_2 = route_stops[best_option[0]][selected_stop]["sequence"]

                # Store the coordinates of the visited stops for their mapping
                visited_stops = []

                # Iterate over the stops of the selected route
                for stop_id, stop_info in route_stops[best_option[0]].items():
                    # Check if the stop sequence number is between seq_1 and seq_2
                    seq_number = stop_info["sequence"]
                    this_orientation = get_bus_orientation(best_option[0], stop_id)
                    if best_option_orientation in this_orientation and seq_1 <= seq_number <= seq_2:
                        # Append the coordinates of the stop to the visited_stops list
                        lat = stop_info["coordinates"][0]
                        lon = stop_info["coordinates"][1]
                        visited_stops.append((seq_number, (lon, lat)))

                # Sorts the visited stops and gets their coordinates
                visited_stops_sorted = sorted(visited_stops, key=lambda x: x[0])
                visited_stops_sorted_coords = [x[1] for x in visited_stops_sorted]

                # Checks if the stop is a Metro Station (they are stored as a number)
                possible_metro_target_name = is_metro_station(selected_stop, metro_stations_dict)

                if possible_metro_target_name is not None:
                    selected_stop = possible_metro_target_name

                print("")
                if possible_metro_name is not None: # Changes the message
                    print("You will get off the train on {} station after {} minutes and {} seconds.".format(selected_stop, minutes, seconds))
                else:
                    print("You will get off the bus on stop {} after {} minutes and {} seconds.".format(selected_stop, minutes, seconds))

                # Maps the best option to get off the best option's service
                folium.Marker(location=[selected_stop_coords[1], selected_stop_coords[0]], 
                      popup="Mejor opción: bajarse del recorrido {} en la parada {}.".format(best_option[0], selected_stop), 
                      icon=folium.Icon(color='cadetblue', icon='plus')).add_to(m)
                ending_distance = [(selected_path[-1][0], selected_path[-1][1]),(selected_stop_coords[1], selected_stop_coords[0])]
                folium.PolyLine(ending_distance,color='black',dash_array='10').add_to(m)

                # Create a polyline connecting the visited stops
                folium.PolyLine(visited_stops_sorted_coords, color='red').add_to(m)

                # Gets the coordinates for the target stop and target location
                final_stop_coords = (selected_stop_coords[1], selected_stop_coords[0])
                final_location_coords = (target_lat, target_lon)
                
                # Calculates the walking time between the target stop and location
                end_walking_time = walking_travel_time(final_stop_coords, final_location_coords, 5)
                end_delta_time = timedelta(seconds=end_walking_time)
                end_walk_min, end_walk_sec = timedelta_separator(end_delta_time)

                # Time walking to stop + waiting the bus + riding the bus + walking to target destination
                total_time = initial_delta_time + best_option[3] + best_travel_time + end_delta_time
                minutes, seconds = timedelta_separator(total_time)

                # Parses the time for the printing
                destination_time = initial_source_time + total_time
                time_string = timedelta_to_hhmm(destination_time)
                print(f"After that, you need to walk for {end_walk_min} minutes and {end_walk_sec} seconds to arrive at the target spot.")
                print(f"Total travel time: {minutes} minutes, {seconds} seconds. You will arrive your destination at {time_string}.")

        # Set the optimal zoom level for the map
        fit_bounds(selected_path, m)

        return m
    else:
        # Empty return
        return


def csa_commands():
    """
    Process the inputs given by the user to run the Connection Scan Algorithm.
    """

    # System's date and time
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    #print("Fecha y hora actuales =", dt_string)

    # Date formatting
    today = date.today()
    today_format = today.strftime("%d/%m/%Y")

    # Time formatting
    moment = now.strftime("%H:%M:%S")
    used_time = datetime.strptime(moment, "%H:%M:%S").time()

    # User inputs
    # Date and time
    source_date = input(
        "Enter the travel's date, in DD/MM/YYY format (press Enter to use today's date) : ") or today_format
    print(source_date)
    source_hour = input(
        "Enter the travel's start time, in HH:MM:SS format (press Enter to start now) : ") or used_time
    if source_hour != used_time:
        source_hour = datetime.strptime(source_hour, "%H:%M:%S").time()
    print(source_hour)

    # Source address
    source_example = "Beauchef 850, Santiago"
    while True:
        source_address = input(
            "Enter the starting point's address, in 'Street #No, Province' format (Ex: 'Beauchef 850, Santiago'):") or source_example
        if source_address.strip() != '':
            break

    # Destination address
    destination_example = "Campus Antumapu Universidad de Chile, Santiago"
    while True:
        target_address = input(
            "Enter the ending point's address, in 'Street #No, Province' format (Ex: 'Campus Antumapu Universidad de Chile, Santiago'):")or destination_example
        if target_address.strip() != '':
            break
    
    start = tm.time() # To calculate the execution time

    # You can change the final number (the margin) as you please. Bigger numbers increase the range for near stops
    # But bigger numbers imply bigger execution times
    best_route_map = connection_scan(source_address, target_address, source_hour, source_date, 0.15)
    
    if not best_route_map:
        print("")
        print("Something went wrong. Please try again later.")
        return
    
    # Displays the results and return
    #display(best_route_map)
    end = tm.time()
    exec_time = round((end-start) / 60,3)
    print("EXECUTION COMPLETED. TIME: {} MINUTES".format(exec_time)) 
    return best_route_map

csa_commands()

# List of addresses to test

#avenida libertador bernardo o'higgins 5121, santiago
#avenida libertador bernardo o'higgins 1460, santiago

#Avenida Independencia 5799, Conchalí
#Avenida Libertador Bernardo O'Higgins 1031, Santiago

Enter the travel's date, in DD/MM/YYY format (press Enter to use today's date) : 
03/08/2023
Enter the travel's start time, in HH:MM:SS format (press Enter to start now) : 
18:32:04
Enter the starting point's address, in 'Street #No, Province' format (Ex: 'Beauchef 850, Santiago'):boriquen 146 maipu
Enter the ending point's address, in 'Street #No, Province' format (Ex: 'Campus Antumapu Universidad de Chile, Santiago'):calle laguna verde oriente 31 maipu
Both addresses have been found.
Processing...

Routes have been found.
Calculating the best route and getting the arrival times for the next buses...

To go from: Boriquen, Las Palmas de Maipú, Maipú, Provincia de Santiago, Región Metropolitana de Santiago, 9290386, Chile
To: Calle Laguna Verde Oriente, Los Laureles de Maipú, Maipú, Provincia de Santiago, Región Metropolitana de Santiago, 9250678, Chile

The best option is to walk for 2 minutes and 18 seconds to stop PI1597, and take the route I08.
The next bus arrives at 18:36.
The ot

In [65]:

# TESTING AREA #
